In [1]:
] activate

 Activating environment at `/tank/home/ciaran/Dropbox/EMAC/DCSideBatteryModeling/Project.toml`


In [82]:
using LinearAlgebra
using ElectricalEngineering
using Interpolations

using OrdinaryDiffEq #Gets the solvers
using Plots
plotly()
using PowerSystems
using DelimitedFiles

include(joinpath(pwd(), "DCSideBatteryModeling", "DCSideBatteryModeling.jl"))

In [3]:
include(joinpath(pwd(), "data","make_data.jl"))

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/pm_io/common.jl:26
┌ Info: Constructing System from Power Models
│   data["name"] = omib
│   data["source_type"] = pti
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:39
┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /home/ciaran/.julia/packages/InfrastructureSystems/WC1sp/src/in_memory_time_series_storage.jl:24
┌ Info: Reading bus data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:140
┌ Info: Reading generator data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:390
┌ Info: Reading branch data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:543
┌ Info: Reading branch data
└ @

LoadError: UndefKeywordError: keyword argument basepower not assigned

In [4]:
omib_sys = System(joinpath(pwd(), "data", "OMIB_inverterDCside.json"))

ErrorException: type NamedTuple has no field validation_descriptor_file

In [47]:
Ub = 480 # Get using PSY
fb = 60 # Get using PSY.
_ωb = 2 * pi * fb
Sb = 0.2e6 # Get using PSY
Vb = 480 # Get using PSY
# System base for AC side
Ib = Sb / Ub
Zb = Vb / Ib
Lb = Zb / _ωb
Cb = 1 / (Zb * _ωb)
# System base for DC side
Vb_dc = (sqrt(8)/sqrt(3))*Vb
Ib_dc = Sb / Vb_dc
Zb_dc = Vb_dc / Ib_dc
Lb_dc = Zb_dc / _ωb
Cb_dc = 1 / (Zb_dc * _ωb)

0.0008634708284065503

In [48]:
parameter_mapping = instantiate_parameters(omib_sys)
M_4th= instantiate_4th_order_model(omib_sys)
M_2nd= instantiate_2nd_order_model(omib_sys)
M_0th= instantiate_0th_order_model(omib_sys)
#u0 = M(parameter_mapping)

ModelOperatingPoint


In [49]:
_parameter_values = [x.second for x in parameter_mapping];

In [50]:
cap_size = 2000
_parameter_values[38]=cap_size*1e-6/Cb_dc
_parameter_values[44]=0

0

In [51]:
M_4th(_parameter_values)
M_2nd(_parameter_values)
M_0th(_parameter_values)

21-element Array{Float64,1}:
  1.0067552230867676
 -0.04029913889433856
  0.20330299948998976
  0.05885597151382374
  0.2003031078414946
 -0.01608748059654234
  0.20230451163495683
  0.008124112353003962
  0.00027215096174116113
 -2.1857989940954274e-5
  0.0704451141318348
 -0.0028057741943914045
  1.084409521544636
  0.4290526993758472
  3.428801320060156e-18
  0.05466665932784704
 -2.482711226656589e-32
  4.600042785237129e-9
 -4.70197740328915e-38
  0.0
  1.437513370386603e-13

In [73]:
disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7

sim_4th = instantiate_4th_order_dae(disturbance, M_4th; tspan = (0.0, 0.05))
sol_4th = solve(sim_4th, Rodas5(), saveat = 0.0001);

In [83]:
sim_2nd = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.05))
sol_2nd = solve(sim_2nd, Rodas5(), saveat = 0.0001);

In [75]:
sim_0th = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.05))
sol_0th = solve(sim_0th, Rodas5(), saveat = 0.0001);

In [85]:
plot(sol_4th, vars=(0,13), label="4")
plot!(sol_2nd, vars=(0,13), label="2")
plot!(sol_0th, vars=(0,13), label="0")

<!DOCTYPE html>
 
 
 Plots.jl

In [86]:
untuned_4th = [sol_4th.t transpose(Array(sol_4th))]
untuned_2nd = [sol_2nd.t transpose(Array(sol_2nd))]
untuned_0th = [sol_0th.t transpose(Array(sol_0th))];

In [87]:
writedlm( "results/dae_sim/untuned_4th_2mF.csv",  untuned_4th, ',')
writedlm( "results/dae_sim/untuned_2nd_2mF.csv",  untuned_2nd, ',')
writedlm( "results/dae_sim/untuned_0th_2mF.csv",  untuned_0th, ',')

In [88]:
gains = readdlm("results/stable_gains/capSize_2000uF.csv", ',')
optIndex = readdlm( "results/optimal_gains/capSize_2000uF.csv", ',')

2×1 Array{Float64,2}:
 4443.0
    0.06802762588059204

In [89]:
_parameter_values[40]=gains[Int(optIndex[1]),1] #K_p^v
_parameter_values[41]=gains[Int(optIndex[1]),2] #K_i^v
_parameter_values[42]=gains[Int(optIndex[1]),3] #K_p^i
_parameter_values[43]=gains[Int(optIndex[1]),4] #K_i^i
_parameter_values[44]=0 #K_pred

0

In [90]:
_parameter_values[40:43]

4-element Array{Float64,1}:
  2.75
  9.75
  3.25
 10.25

In [91]:
M_4th(_parameter_values)
M_2nd(_parameter_values)
M_0th(_parameter_values)

21-element Array{Float64,1}:
  1.0067552230867676
 -0.04029913889433857
  0.20330299948998976
  0.05885597151382373
  0.20030310784149463
 -0.01608748059654234
  0.20230451163495683
  0.008124112353003962
  0.00027215096174116113
 -2.185798994095429e-5
  0.0704451141318348
 -0.002805774194391405
  1.084409521544636
  0.4290526993758472
  5.820756158934316e-19
  0.0551466592634089
 -7.779704729421164e-41
  4.600042785237128e-9
  0.0
  0.0
  1.437513370386603e-13

In [92]:
disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7

sim_4th = instantiate_4th_order_dae(disturbance, M_4th; tspan = (0.0, 0.02))
sol_4th = solve(sim_4th, Rodas5(), saveat = 0.0001);

In [93]:
sim_2nd = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.02))
sol_2nd = solve(sim_2nd, Rodas5(), saveat = 0.0001);

In [94]:
sim_0th = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.02))
sol_0th = solve(sim_0th, Rodas5(), saveat = 0.0001);

In [95]:
plot(sol_4th, vars=(0,13))
plot!(sol_2nd, vars=(0,13))
plot!(sol_0th, vars=(0,13))

<!DOCTYPE html>
 
 
 Plots.jl

In [96]:
tuned_4th = [sol_4th.t transpose(Array(sol_4th))]
tuned_2nd = [sol_2nd.t transpose(Array(sol_2nd))]
tuned_0th = [sol_0th.t transpose(Array(sol_0th))];

In [97]:
writedlm("results/dae_sim/tuned_4th_2mF.csv",  tuned_4th, ',')
writedlm("results/dae_sim/tuned_2nd_2mF.csv",  tuned_2nd, ',')
writedlm("results/dae_sim/tuned_0th_2mF.csv",  tuned_0th, ',')

In [98]:
_parameter_values[44]=0 #K_pred
M_0th(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 #Load

sim_k0 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.03))
sol_k0 = solve(sim_k0, Rodas5(), saveat = 0.0001);

In [99]:
_parameter_values[44]=1 #K_pred
M_0th(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 #Load

sim_k1 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.02))
sol_k1 = solve(sim_k1, Rodas5(), saveat = 0.0001);

In [100]:
_parameter_values[44]=2 #K_pred
M_0th(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 #Load

sim_k2 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.03))
sol_k2 = solve(sim_k2, Rodas5(), saveat = 0.0001);

In [101]:
_parameter_values[44]=2.5 #K_pred
M_0th(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 #Load

sim_k25 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.03))
sol_k25 = solve(sim_k25, Rodas5(), saveat = 0.0001);

In [102]:
_parameter_values[44]=5 #K_pred
M_0th(_parameter_values)

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 #Load

sim_k5 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.03))
sol_k5 = solve(sim_k5, Rodas5(), saveat = 0.0001);

In [103]:
plot(sol_k0, vars=(0,13), xlims = (0,0.025), label="0")
plot!(sol_k1, vars=(0,13), xlims = (0,0.025), label="1")
plot!(sol_k2, vars=(0,13), xlims = (0,0.025), label="2")
plot!(sol_k25, vars=(0,13), xlims = (0,0.025), label="2.5")
plot!(sol_k5, vars=(0,13), xlims = (0,0.025), label="5")

<!DOCTYPE html>
 
 
 Plots.jl

In [110]:
kpred_0 = [sol_k0.t transpose(Array(sol_k0))]
kpred_1 = [sol_k1.t transpose(Array(sol_k1))]
kpred_2 = [sol_k2.t transpose(Array(sol_k2))];
kpred_25 = [sol_k25.t transpose(Array(sol_k25))]
kpred_5 = [sol_k5.t transpose(Array(sol_k5))];

In [111]:
writedlm( "results/dae_sim/kpred_0.csv",  kpred_0, ',')
writedlm( "results/dae_sim/kpred_1.csv",  kpred_1, ',')
writedlm( "results/dae_sim/kpred_2.csv",  kpred_2, ',')
writedlm( "results/dae_sim/kpred_25.csv",  kpred_25, ',')
writedlm( "results/dae_sim/kpred_5.csv",  kpred_5, ',')

In [115]:
nadir_reduction = (minimum(kpred_2[:,14])-minimum(kpred_0[:,14]))/(1.1-minimum(kpred_0[:,14]))

0.08724367918680817

In [116]:
nadir_reduction

0.08724367918680817

In [114]:
_parameter_values[38]

2.3162334316386826

## Generating Heatmap

In [103]:
heat_map_values=reshape([],0,3);
cap_range = 2000:500:7000
kpred_range = 0:0.25:5
for cap_size in cap_range
    println(cap_size)
    for k in kpred_range
        _parameter_values[38]=cap_size*1e-6/Cb_dc
        fileName=string("results/stable_gains/capSize_", cap_size, "uF.csv")
        gains = readdlm(fileName, ',', Float64)
        fileName=string("results/optimal_gains/capSize_", cap_size, "uF.csv")
        optIndex = readdlm(fileName, ',', Float64)
        _parameter_values[40]=gains[Int(optIndex[1]),1] #K_p^v
        _parameter_values[41]=gains[Int(optIndex[1]),2] #K_i^v
        _parameter_values[42]=gains[Int(optIndex[1]),3] #K_p^i
        _parameter_values[43]=gains[Int(optIndex[1]),4] #K_i^i
        _parameter_values[44]=k #K_pred
        M_0th(_parameter_values)

        disturbance=deepcopy(_parameter_values)
        disturbance[9]=0.7

        sim = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.02))
        sol = solve(sim, Rodas5(), saveat = 0.0001);
        solArray = Array(sol)
        nadir=100*maximum(_parameter_values[39].-solArray[13,:])
        
        heat_map_values = [heat_map_values; [cap_size/1e3 k round(nadir, digits=3)]]
    end
end
writedlm( "results/dae_sim/heat_map_matrix.csv",  heat_map_values, ',')

2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000


In [104]:
writedlm( "results/dae_sim/heat_map_matrix.dat",  heat_map_values)